# Low-latency item-to-item recommendation system - Creating ANN index

## Overview

This notebook is a part of the series that describes the process of implementing a [**Low-latency item-to-item recommendation system**](https://github.com/jarokaz/analytics-componentized-patterns/tree/master/retail/recommendation-system/bqml-scann).

The notebook demonstrates how to create and deploy an ANN index using embeddings created in the preceding notebooks. In this notebook you go through the following steps.

1. Exporting embeddings from BigQuery into the JSONL formated file.
2. Creating an ANN Index using the exported embeddings.
3. Creating and ANN Endpoint. 
4. Deploying the ANN Index to the ANN Endpoint.
5. Testing the deployed ANN Index.

This notebook was designed to run on [AI Platform Notebooks](https://cloud.google.com/ai-platform/notebooks/docs) using the standard TensorFlow 2.3 image. Your notebook instance should be in the same project as the AI Platform ANN Service.

While AI Platform ANN Service is in the Experimental stage your project must be allow-listed before using the service. Contact `gcp-ann-feedback@` to allow list your project and user id.

## Setting up the notebook's environment

### Configuring AI Platform ANN Service

#### Enable the required Cloud APIs

You need to enable the following APIs to use the ANN service:
* aiplatform.googleapis.com
* servicenetworking.googleapis.com
* compute.googleapis.com

#### Prepare a VPC network

In the experimental release, ANN service is only accessible using private endpoints. Before using the service you need to have a [VPC network](https://cloud.google.com/vpc) configured with [private services access](https://cloud.google.com/vpc/docs/configure-private-services-access). You can use the `default` VPC or create a new one.

The below instructions are for a VPC that was created with auto subnets and regional dynamic routing mode (defaults). It is recommended that you execute the below commands from Cloud Shell, using the account with appropriate permissions - `roles/compute.networkAdmin`.

1. Set environment variables for your project ID, the name of your VPC network, and the name of your reserved range of addresses. The name of the reserved range can be an arbitrary name. It is for display only.

```
PROJECT_ID=<your-project-id>
gcloud config set project $PROJECT_ID
NETWORK_NAME=<your-VPC-network-name>
PEERING_RANGE_NAME=google-reserved-range

```

2. Reserve an IP range for Google services. The reserved range should be large enought to accommodate all peered services. The below command reserves a CIDR block with mask /16

```
gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service: AI Platform Online Prediction" \
  --network=$NETWORK_NAME \
  --purpose=VPC_PEERING \
  --project=$PROJECT_ID

```

3. Create a private connection to establish a VPC Network Peering between your VPC network and the Google services network.

```
gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK_NAME \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

```


### Importing notebook dependencies

In [1]:
import base64
import datetime
import logging
import os
import json
import pandas as pd
import time
import sys

import grpc

import google.auth
import numpy as np
import tensorflow.io as tf_io

from google.cloud import bigquery
from typing import List, Optional, Text, Tuple

In the experimental release, the Online Querying API of the deployed ANN index is exposed throught the GRPC interface. The `ann_grpc` folder contains the grpc client stub to interface to the service.

In [2]:
ANN_GRPC_ENDPOINT_STUB = 'ann_grpc'
if ANN_GRPC_ENDPOINT_STUB not in sys.path:
    sys.path.append(ANN_GRPC_ENDPOINT_STUB)

In [3]:
import ann_grpc.match_pb2_grpc as match_pb2_grpc
import ann_grpc.match_pb2 as match_pb2

### Configure GCP environment

Set the following constants to the values reflecting your environment:

* `PROJECT_ID` - your GCP project ID.
* `PROJECT_NUMBER` - your GCP project number.
* `BQ_DATASET_NAME` - the name of the BigQuery dataset that contains the item embeddings table.
* `BQ_LOCATION` - the dataset location
* `DATA_LOCATION` - a GCS location for the exported embeddings (JSONL) files.
* `VPC_NAME` - a name of the GCP VPC to use for the index deployments. Use the name of the VPC prepared in the previous section. 
* `REGION` - a compute region. Don't change the default - `us-central` - while the ANN Service is in the experimental stage


In [4]:
PROJECT_ID = 'jk-mlops-dev' # <-CHANGE THIS
PROJECT_NUMBER = '895222332033' # <-CHANGE THIS
BQ_DATASET_NAME = 'song_embeddings' # <- CHANGE THIS
BQ_LOCATION = 'US' # <- CHANGE THIS
DATA_LOCATION = 'gs://jk-ann-staging/embeddings' # <-CHANGE THIS
VPC_NAME = 'default' # <-CHANGE THIS

EMBEDDINGS_TABLE = 'item_embeddings'
REGION = 'us-central1'
MATCH_SERVICE_PORT = 10000

## Exporting the embeddings

In the preceeding notebookes you trained the Matrix Factorization BQML model and exported the embeddings to the `item_embeddings` table. 

In this step you will extract the embeddings to a set of JSONL files in the format required by the ANN service.

### Verify the number of embeddings

In [5]:
client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

In [6]:
query = f"""
    SELECT COUNT(*) embedding_count
    FROM {BQ_DATASET_NAME}.item_embeddings;
"""

query_job = client.query(query)
query_job.to_dataframe()

,embedding_count
0,35519


### Export the embeddings

In [7]:
file_name_pattern = 'embedding-*.json'
destination_uri = f'{DATA_LOCATION}/{file_name_pattern}'
table_id = 'item_embeddings'
destination_format = 'NEWLINE_DELIMITED_JSON'

dataset_ref = bigquery.DatasetReference(PROJECT_ID, BQ_DATASET_NAME)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.job.ExtractJobConfig()
job_config.destination_format = bigquery.DestinationFormat.NEWLINE_DELIMITED_JSON

extract_job = client.extract_table(
    table_ref,
    destination_uris=destination_uri,
    job_config=job_config,
    #location=BQ_LOCATION,
)  

extract_job.result()

Inspect the extracted files.

In [8]:
! gsutil ls {DATA_LOCATION}

gs://jk-ann-staging/embeddings/embedding-000000000000.json


## Creating an ANN index deployment

Deploying an ANN index is a 3 step process:
1. Creating an index from source files
2. Creating an endpoint to access the index
3. Deploying the index to the endpoint


You will use the REST interface to invoke the AI Platform ANN Service Control Plane API that manages indexes, endpoints, and deployments.

After the index has been deployed you can submit matching requests using Online Querying API. In the experimental stage this API is only accessible through the gRPC interface.

Refer to [ANN service user guide](https://docs.google.com/document/d/1D5HsS829UuYUuyCTHWSFqR8dl4KNvL9C82nEs9vLtjg/edit) for more information about the ANN Service APIs.

### Define helper classes to encapsulate the ANN Service REST API.

Currently, there is no Python client that encapsulates the ANN Service Control Plane API. The below code snippet defines a simple wrapper that encapsulates a subset of REST APIs used in this notebook.



In [9]:
class ANNClient(object):
    """Base ANN Service client."""
    
    def __init__(self, project_id, project_number, region):
        credentials, _ = google.auth.default()
        self.authed_session = google.auth.transport.requests.AuthorizedSession(credentials)
        self.ann_endpoint = f'{region}-aiplatform.googleapis.com'
        self.ann_parent = f'https://{self.ann_endpoint}/v1alpha1/projects/{project_id}/locations/{region}'
        self.project_id = project_id
        self.project_number = project_number
        self.region = region
        
    def wait_for_completion(self, operation_id, message, sleep_time):
        """Waits for a completion of a long running operation."""
        
        api_url = f'{self.ann_parent}/operations/{operation_id}'

        start_time = datetime.datetime.utcnow()
        while True:
            response = self.authed_session.get(api_url)
            if 'done' in response.json().keys():
                logging.info('Operation completed!')
                break
            elapsed_time = datetime.datetime.utcnow() - start_time
            logging.info('{}. Elapsed time since start: {}.'.format(
                message, str(elapsed_time)))
            time.sleep(sleep_time)
    
        return response.json()['response']


class IndexClient(ANNClient):
    """Encapsulates a subset of control plane APIs 
    that manage ANN indexes."""

    def __init__(self, project_id, project_number, region):
        super().__init__(project_id, project_number, region)

    def create_index(self, display_name, description, metadata):
        """Creates an ANN Index."""
    
        api_url = f'{self.ann_parent}/indexes'
    
        request_body = {
            'display_name': display_name,
            'description': description,
            'metadata': metadata
        }
    
        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
        
        return operation_id

    def list_indexes(self, display_name=None):
        """Lists all indexes with a given display name or
        all indexes if the display_name is not provided."""
    
        if display_name:
            api_url = f'{self.ann_parent}/indexes?filter=display_name="{display_name}"'
        else:
            api_url = f'{self.ann_parent}/indexes'

        response = self.authed_session.get(api_url).json()

        return response['indexes'] if response else []
    
    def delete_index(self, index_id):
        """Deletes an ANN index."""
        
        api_url = f'{self.ann_parent}/indexes/{index_id}'
        response = self.authed_session.delete(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)


class IndexDeploymentClient(ANNClient):
    """Encapsulates a subset of control plane APIs 
    that manage ANN endpoints and deployments."""
    
    def __init__(self, project_id, project_number, region):
        super().__init__(project_id, project_number, region)

    def create_endpoint(self, display_name, vpc_name):
        """Creates an ANN endpoint."""
    
        api_url = f'{self.ann_parent}/indexEndpoints'
        network_name = f'projects/{self.project_number}/global/networks/{vpc_name}'

        request_body = {
            'display_name': display_name,
            'network': network_name
        }

        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
    
        return operation_id
    
    def list_endpoints(self, display_name=None):
        """Lists all ANN endpoints with a given display name or
        all endpoints in the project if the display_name is not provided."""
        
        if display_name:
            api_url = f'{self.ann_parent}/indexEndpoints?filter=display_name="{display_name}"'
        else:
            api_url = f'{self.ann_parent}/indexEndpoints'

        response = self.authed_session.get(api_url).json()
 
        return response['indexEndpoints'] if response else []
    
    def delete_endpoint(self, endpoint_id):
        """Deletes an ANN endpoint."""
        
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}'
        
        response = self.authed_session.delete(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)
        
        return response.json()
    
    def create_deployment(self, display_name, deployment_id, endpoint_id, index_id):
        """Deploys an ANN index to an endpoint."""
    
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}:deployIndex'
        index_name = f'projects/{self.project_number}/locations/{self.region}/indexes/{index_id}'

        request_body = {
            'deployed_index': {
                'id': deployment_id,
                'index': index_name,
                'display_name': display_name
            }
        }

        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
        
        return operation_id
    
    def get_deployment_grpc_ip(self, endpoint_id, deployment_id):
        """Returns a private IP address for a gRPC interface to 
        an Index deployment."""
  
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}'

        response = self.authed_session.get(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)
            
        endpoint_ip = None
        if 'deployedIndexes' in response.json().keys():
            for deployment in response.json()['deployedIndexes']:
                if deployment['id'] == deployment_id:
                    endpoint_ip = deployment['privateEndpoints']['matchGrpcAddress']
                    
        return endpoint_ip

    
    def delete_deployment(self, endpoint_id, deployment_id):
        """Undeployes an index from an endpoint."""
        
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}:undeployIndex'
        
        request_body = {
            'deployed_index_id': deployment_id
        }
    
        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        
        return response
    
    
index_client = IndexClient(PROJECT_ID, PROJECT_NUMBER, REGION)
deployment_client = IndexDeploymentClient(PROJECT_ID, PROJECT_NUMBER, REGION)

### Create an index

#### List all indexes registered with the ANN service

In [22]:
indexes = index_client.list_indexes()

if not indexes:
    print('There are not any indexes registered with the service')

for index in indexes:
    print(index['name'])

projects/895222332033/locations/us-central1/indexes/2779283920041017344
projects/895222332033/locations/us-central1/indexes/6418192418956378112
projects/895222332033/locations/us-central1/indexes/4720335359437701120


#### Configure and create a new index based on the exported embeddings

Index creation is a long running operation. Be patient.

In [ ]:
index_display_name = 'Song embeddings'
index_description = 'Song embeddings created BQML Matrix Factorization model'

index_metadata = {
    'contents_delta_uri': DATA_LOCATION,
    'config': {
        'dimensions': 50,
        'approximate_neighbors_count': 50,
        'distance_measure_type': 'DOT_PRODUCT_DISTANCE',
        'feature_norm_type': 'UNIT_L2_NORM',
        'tree_ah_config': {
            'child_node_count': 1000,
            'max_leaves_to_search': 100
         }
    }
}

In [ ]:
logging.getLogger().setLevel(logging.INFO)

operation_id = index_client.create_index(index_display_name, 
                                          index_description,
                                          index_metadata)

response = index_client.wait_for_completion(operation_id, 'Creating index', 20)
print(response)

#### Verify that the index was created

In [ ]:
indexes = index_client.list_indexes(index_display_name)

for index in indexes:
    print(index['name'])

if indexes: 
    index_id = index['name'].split('/')[-1]
    print(f'Index: {index_id} will be used for deployment')
else:
    print('No indexes available for deployment')

### Create the index deployment

#### List all endpoints registered with the ANN service

In [20]:
endpoints = deployment_client.list_endpoints()

if not endpoints:
    print('There are not any endpoints registered with the service')

for endpoint in endpoints:
    print(endpoint['name'])

projects/895222332033/locations/us-central1/indexEndpoints/1908400342098247680


#### Create an index endpoint

In [16]:
deployment_display_name = 'Song embeddings endpoint'

In [ ]:
operation_id = deployment_client.create_endpoint(deployment_display_name, VPC_NAME)

response = index_client.wait_for_completion(operation_id, 'Waiting for endpoint', 10)
print(response)

#### Verify that the endpoint was created

In [17]:
endpoints = deployment_client.list_endpoints(deployment_display_name)

for endpoint in endpoints:
    print(endpoint['name'])
    
if endpoints: 
    endpoint_id = endpoint['name'].split('/')[-1]
    print(f'Endpoint: {endpoint_id} will be used for deployment')
else:
    print('No endpoints available for deployment')


No endpoints available for deployment


#### Deploy the index to the endpoint

##### Set the deployment ID

The ID must be unique within your project

In [ ]:
deployment_display_name = 'Song embeddings deployed index'
deployed_index_id = 'songs_embeddings_deployed'

##### Deploy the index

Be patient. Index deployment is a long running operation

In [ ]:
operation_id = deployment_client.create_deployment(deployment_display_name, 
                                                   deployed_index_id,
                                                   endpoint_id,
                                                   index_id)

response = index_client.wait_for_completion(operation_id, 'Waiting for deployment', 10)
print(response)

#### Retrieve the gRPC private endpoint for the ANN Match service

In [ ]:
deployed_index_ip = deployment_client.get_deployment_grpc_ip(endpoint_id, deployed_index_id)
endpoint = f'{deployed_index_ip}:{MATCH_SERVICE_PORT}'
print(f'gRPC endpoint for the: {deployed_index_id} deployment is: {endpoint}')

## Querying the ANN service

You will use the gRPC interface to query the deployed index.

### Create a helper wrapper around the Match Service gRPC API.

The wrapper uses the pre-generated gRPC stub to the Online Querying gRPC interface. 

In [ ]:
class MatchService(object):
    """This is a wrapper around Online Querying gRPC interface."""
    def __init__(self, endpoint, deployed_index_id):
        self.endpoint = endpoint
        self.deployed_index_id = deployed_index_id

    def single_match(
        self,
        embedding: List[float],
        num_neighbors: int) -> List[Tuple[str, float]]:
        """Requests a match for a single embedding."""
    
        match_request = match_pb2.MatchRequest(deployed_index_id=self.deployed_index_id,
                                               float_val=embedding,
                                               num_neighbors=num_neighbors)
        with grpc.insecure_channel(endpoint) as channel:
            stub = match_pb2_grpc.MatchServiceStub(channel)
            response = stub.Match(match_request)
    
        return [(neighbor.id, neighbor.distance) for neighbor in response.neighbor]


    def batch_match(
        self,
        embeddings: List[List[float]],
        num_neighbors: int) -> List[List[Tuple[str, float]]]:
        """Requests matches ofr a list of embeddings."""
    
        match_requests = [
            match_pb2.MatchRequest(deployed_index_id=self.deployed_index_id,
                                   float_val=embedding,
                                   num_neighbors=num_neighbors)
            for embedding in embeddings]
    
        batches_per_index = [
            match_pb2.BatchMatchRequest.BatchMatchRequestPerIndex(
                deployed_index_id=self.deployed_index_id,
                requests=match_requests)]
    
        batch_match_request = match_pb2.BatchMatchRequest(
            requests=batches_per_index)
    
        with grpc.insecure_channel(endpoint) as channel:
            stub = match_pb2_grpc.MatchServiceStub(channel)
            response = stub.BatchMatch(batch_match_request)
        
        matches = []
        for batch_per_index in response.responses:
            for match in batch_per_index.responses:
                matches.append(
                    [(neighbor.id, neighbor.distance) for neighbor in match.neighbor])
        
        return matches
    
match_service = MatchService(endpoint, deployed_index_id)

### Prepare sample data

Retrieve a few embeddings from the BigQuery embedding table

In [ ]:
%%bigquery df_embeddings

SELECT id, embedding
FROM `recommendations.item_embeddings` 
LIMIT 10

In [ ]:
sample_embeddings = [list(embedding) for embedding in df_embeddings['embedding']]
sample_embeddings[0]

### Run a single match query

The following call requests 10 closest neighbours for a single embedding.

In [ ]:
%%time 

single_match = match_service.single_match(sample_embeddings[0], 10)
single_match

### Run a batch match query

The following call requests 3 closest neighbours for each of the embeddings in a batch of 5.

In [ ]:
%%time 

batch_match = match_service.batch_match(sample_embeddings[0:5], 3)
batch_match

## Clean up

**WARNING**

The below code will delete all ANN deployments, endpoints, and indexes in the configured project.

### Delete index deployments and endpoints

In [11]:
for endpoint in deployment_client.list_endpoints():
    endpoint_id = endpoint['name'].split('/')[-1]
    if 'deployedIndexes' in endpoint.keys():
        for deployment in endpoint['deployedIndexes']:
            print('   Deleting index deployment: {} in the endpoint: {} '.format(deployment['id'], endpoint_id))
            deployment_client.delete_deployment(endpoint_id, deployment['id'])
    print('Deleting endpoint: {}'.format(endpoint['name']))
    deployment_client.delete_endpoint(endpoint_id)

### Delete indexes

In [12]:
for index in index_client.list_indexes():
    index_id = index['name'].split('/')[-1]
    print('Deleting index: {}'.format(index['name']))
    index_client.delete_index(index_id)

## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**